In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Clone MMDetection from the main branch (or specify a valid tag if needed)
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info
  No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
  running egg_info
  creating /tmp/pip-pip-egg-info-iu5g1wmw/mmdet.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-iu5g1wmw/mmdet.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-iu5g1wmw/mmdet.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Removing file or directory /usr/local/lib/python3.11/dist-packages/mmdet.egg-link
      Removing pth entries from /usr/local/lib/python3.11/dist-pa

In [ ]:
!pip install -U openmim
!mim install mmengine
!mim install "mmcv==2.1.0"

Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 15.3 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.4/471.4 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!mkdir -p configs/faster_rcnn
!wget https://raw.githubusercontent.com/open-mmlab/mmdetection/main/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py -O configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py

--2025-04-23 08:52:33--  https://raw.githubusercontent.com/open-mmlab/mmdetection/main/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177 [text/plain]
Saving to: ‘configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py’

configs/faster_rcnn 100%[===================>]     177  --.-KB/s    in 0s      

2025-04-23 08:52:33 (7.17 MB/s) - ‘configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py’ saved [177/177]



In [3]:
# !git clone -b 3.3.0 https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# !pip install -v -e .

fatal: destination path 'mmdetection' already exists and is not an empty directory.
/content/mmdetection
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/mmdetection
  Running command python setup.py egg_info
  No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
  running egg_info
  creating /tmp/pip-pip-egg-info-iqu0nkve/mmdet.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-iqu0nkve/mmdet.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-iqu0nkve/mmdet.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Removing file or directory /usr/local/lib/python3.11/dist-packages/mmdet.egg-link
      Removing pth entries from /usr/local/lib/python3.11/dist-packages/easy-install.pth:
      Removing entry: /content/mmdetection
      Successfully uninstalled mmdet-3.3.0
  Running setup.py

In [4]:
!pip install -q pycocotools scikit-learn tqdm

In [5]:
import mmengine
print(mmengine.__version__)

0.10.7


In [13]:
###### 2. New Code Cell ######
# Utility functions

import mmdet
import os, json, torch, numpy as np, matplotlib.pyplot as plt
from mmengine.config import Config
from mmengine.registry import init_default_scope  # ✅ Correct import
from mmdet.apis import init_detector, inference_detector, build_detector
from pycocotools.coco import COCO
from tqdm import tqdm
torch.set_num_threads(4)

###### Change between cpu and cuda based off your system #######
def load_model(cfg_path, checkpoint_path, device='cpu'):        # ① default to CPU
    cfg = Config.fromfile(cfg_path)

    model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

    checkpoint = torch.load(
        checkpoint_path,
        map_location=device,    # ② send tensors straight to the CPU
        weights_only=False
    )

    model.load_state_dict(checkpoint['state_dict'])
    model.to(device).eval()    # ③ put the model on the CPU
    return model

def calc_iou(b1, b2):
    """Axis‑aligned IoU for two [x1,y1,x2,y2] boxes."""
    x1 = np.maximum(b1[0], b2[0])
    y1 = np.maximum(b1[1], b2[1])
    x2 = np.minimum(b1[2], b2[2])
    y2 = np.minimum(b1[3], b2[3])
    inter = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    area1 = (b1[2]-b1[0]) * (b1[3]-b1[1])
    area2 = (b2[2]-b2[0]) * (b2[3]-b2[1])
    union = area1 + area2 - inter
    return inter / union if union else 0

def classify_predictions(pred_boxes, pred_scores, gt_boxes, iou_thr=0.5):
    """Return mask of TP for each prediction (FP otherwise)."""
    matched_gt = set()
    tps = []
    for box, score in sorted(zip(pred_boxes, pred_scores),
                             key=lambda x: -x[1]):    # high→low
        found = False
        for i, g in enumerate(gt_boxes):
            if i in matched_gt:         # already matched
                continue
            if calc_iou(box, g) >= iou_thr:
                matched_gt.add(i)
                found = True
                break
        tps.append(found)
    return np.array(tps, dtype=bool)


ImportError: cannot import name 'build_detector' from 'mmdet.apis' (/content/mmdetection/mmdet/apis/__init__.py)

In [7]:
###### 3. New Code Cell ######
# Run inference & gather statistics

drive_root = '/content/drive/MyDrive'
dataset_root = os.path.join(drive_root, '7_coco')

# << edit ONLY these three lines as needed >>
# === Paths (Google Drive Mounted) ===

cfg_path = '/content/mmdetection/configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py'
# cfg_path = Config.fromfile('configs/faster_rcnn/faster-rcnn_r50_fpn_1x_coco.py')
checkpoint_path = os.path.join(drive_root, 'faster_rcnn_output_2', 'epoch_10.pth')
test_ann         = os.path.join(dataset_root, 'test', 'test_annotations.json')

model   = load_model(cfg_path, checkpoint_path)
coco    = COCO(test_ann)

all_scores = []        # per‑prediction confidence
all_is_tp  = []        # per‑prediction TP mask
num_gt     = 0         # total GT boxes

for img_id in tqdm(coco.getImgIds()):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(dataset_root, img_info['file_name'])
    result   = inference_detector(model, img_path)
    # result is list of ndarray per class; concatenate:
    boxes = np.vstack(result)               # (N,5) where last col is score
    if boxes.size == 0:
        continue
    pred_boxes   = boxes[:, :4]
    pred_scores  = boxes[:, 4]
    gt_anns      = coco.loadAnns(coco.getAnnIds(imgIds=[img_id]))
    gt_boxes     = [ann['bbox'] for ann in gt_anns]          # COCO: x,y,w,h
    gt_boxes     = [[x, y, x+w, y+h] for x,y,w,h in gt_boxes]

    num_gt      += len(gt_boxes)
    tps          = classify_predictions(pred_boxes, pred_scores, gt_boxes)
    all_scores.extend(pred_scores.tolist())
    all_is_tp.extend(tps.tolist())

all_scores = np.array(all_scores)
all_is_tp  = np.array(all_is_tp)

NameError: name 'build_detector' is not defined